In [2]:
import tensorflow as tf
import numpy as np
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
from matplotlib import pyplot as plt
import serial
import serial.tools.list_ports
import time
import io

2022-04-23 19:32:37.295468: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-23 19:32:37.295484: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def _ssd_generate_anchors() -> np.ndarray:
    """
    (reference: mediapipe/calculators/tflite/ssd_anchors_calculator.cc)
    """
    layer_id = 0
    num_layers = 4
    strides = [8, 16, 16, 16]
    assert len(strides) == num_layers
    input_height = 128
    input_width = 128
    anchor_offset_x = 0.5
    anchor_offset_y = 0.5
    interpolated_scale_aspect_ratio = 1
    anchors = []
    while layer_id < num_layers:
        last_same_stride_layer = layer_id
        repeats = 0
        while (last_same_stride_layer < num_layers and
               strides[last_same_stride_layer] == strides[layer_id]):
            last_same_stride_layer += 1
            # aspect_ratios are added twice per iteration
            repeats += 2 if interpolated_scale_aspect_ratio == 1.0 else 1
        stride = strides[layer_id]
        feature_map_height = input_height // stride
        feature_map_width = input_width // stride
        for y in range(feature_map_height):
            y_center = (y + anchor_offset_y) / feature_map_height
            for x in range(feature_map_width):
                x_center = (x + anchor_offset_x) / feature_map_width
                for _ in range(repeats):
                    anchors.append((x_center, y_center))
        layer_id = last_same_stride_layer
    return np.array(anchors, dtype=np.float32)
anchors = _ssd_generate_anchors()

In [4]:
def processImage(image):
    original_h = image.shape[0]
    original_w = image.shape[1]
    
    # RUN FACE DETECT MODEL
    resized_image = np.reshape(tf.image.resize(image, [128,128]) / 255,[1,128,128,3])
    interpreter = tf.lite.Interpreter(model_path="models/face_detection_front.tflite")
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], resized_image)
    interpreter.invoke()
    raw_boxes = interpreter.get_tensor(output_details[0]['index'])
    raw_scores = interpreter.get_tensor(output_details[1]['index'])

    # GET HIGHEST SCORE
    raw_scores[raw_scores < -80] = -80
    raw_scores[raw_scores > 80] = 80
    sigmoid_scores = 1.0 / (1.0 + np.exp(-raw_scores))
    best_score = np.max(sigmoid_scores)
    if best_score < 0.8:
        return 0
    best_index = np.argmax(sigmoid_scores)
    
    
    # DECODE BOX
    scale = 128
    num_points = 8
    # scale all values (applies to positions, width, and height alike)
    face_box_center = raw_boxes[0, best_index, 0:2] / scale
    face_box_wh = raw_boxes[0, best_index, 2:4] / scale
    # adjust center coordinates to anchor positions
    face_box_center += anchors[best_index]
    # convert x_center, y_center, w, h to xmin, ymin, xmax, ymax
    center = np.array(face_box_center)
    half_size = face_box_wh / 2
    [xmin, ymin] = center - half_size
    [xmax, ymax] = center + half_size
    # only full faces in frame
    if (xmin<0): return 0
    if (xmax>original_w): return 0
    if (ymin<0): return 0
    if (ymax>original_h): return 0

    # EXTRACT FACE
    face = image[int(ymin*original_h):int(ymax*original_h), int(xmin*original_h):int(xmax*original_h),:]

    # MOBILENET INTERPRETER
    resized_face_uint8 = np.reshape(tf.image.resize(face, [192,192]),[1,192,192,3]).astype('uint8')
    interpreter = tf.lite.Interpreter(model_path="models/mobilenet_v1_0.25_192_quant.tflite")
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], resized_face_uint8)
    interpreter.invoke()
    logits = interpreter.get_tensor(output_details[0]['index'])
    return logits[0]

In [5]:
# ARDUINO SERIAL
HANDSHAKE = 0

ON_REQUEST = 1
STREAM_IMAGE = 2
STREAM_LOGITS = 3

INITIALIZE = 4 
CAPTURE_IMAGE = 5
FACE_DETECTED = 6
FACE_NOT_DETECTED = 7

TRAIN_1 = 8
TRAIN_2 = 9
CLASSIFY = 10


def find_arduino(port=None):
    """Get the name of the port that is connected to Arduino."""
    if port is None:
        ports = serial.tools.list_ports.comports()
        for p in ports:
            if p.manufacturer is not None and "Arduino" in p.manufacturer:
                port = p.device
    return port


def handshake_arduino(
    arduino, sleep_time=1, print_handshake_message=False, handshake_code=0
):
    """Make sure connection is established by sending
    and receiving bytes."""
    # Close and reopen
    arduino.close()
    arduino.open()

    # Chill out while everything gets set
    time.sleep(sleep_time)

    # Set a long timeout to complete handshake
    timeout = arduino.timeout
    arduino.timeout = 2


    # Send request to Arduino
    arduino.write(bytes([handshake_code]))

    # Read in what Arduino sent
    handshake_message = arduino.read_until()

    # Send and receive request again
    arduino.write(bytes([handshake_code]))
    handshake_message = arduino.read_until()

    # Print the handshake message, if desired
    if print_handshake_message:
        print("Handshake message: " + handshake_message.decode())

    # Reset the timeout
    arduino.timeout = timeout

    
def image_stream(arduino):
    buffer_size = 1000;

    # Receive Chunks
    length = 0
    while length == 0:
        try:
            length = int(arduino.read_until().decode())
        except:
            pass

    # Stream Chunks
    image_bytes = bytearray()
    chunks = length // 1000
    x = 0
    while x < chunks:
        while arduino.inWaiting() < 1000:
            pass
        image_bytes.extend(arduino.read(1000))
        x+=1
    while arduino.inWaiting() < length%1000:
        pass
    image_bytes.extend(arduino.read(length % 1000))
    image_PIL = Image.open(io.BytesIO(image_bytes))
    
    # Square off the image
    width, height = image_PIL.size   # Get dimensions
    left = (width - height)/2
    top = (height - height)/2
    right = (width + height)/2
    bottom = (height + height)/2

    image = image_PIL.crop((left, top, right, bottom))
    image = np.array(image)
    
    # Turn off the stream
    return image

def get_info(arduino):
    # Turn on the stream
    arduino.write(bytes([STATUS]))
    # Read in what Arduino sent
    time.sleep(3)
    msg = arduino.read_all().decode()
    print(msg)

In [11]:
# ESTABLISH COMMUNICATION
port = find_arduino()
arduino = serial.Serial(port, baudrate=250000)
handshake_arduino(arduino, handshake_code=HANDSHAKE, print_handshake_message=True)
_ = arduino.read_all()
arduino.write(bytes([INITIALIZE]))
while True:
    while arduino.inWaiting() == 0:
        pass
    msg = arduino.read_until().decode().rstrip()
    print(msg)
    if (msg == "Camera Initialized" or msg == "Camera Not Initialized"):
        break
    pass

Handshake message: Message received.

Attempting to start Arducam
Camera Initialized


In [12]:
#%%timeit
i = 0
while i < 10:
    # IMAGE CAPTURE AND PROCESS
    arduino.write(bytes([CAPTURE_IMAGE]))
    msgs = 0
    while True:
        try:
            msg = arduino.read_until().decode().rstrip()
            print(msg)
            if (msg == "Camera Not Initialized" or msg == "ReadData failed" or msg == "Finished reading"):
                break
        except:
            pass

    if msg == "Finished reading":
        print('FINISHED')
        image = image_stream(arduino)
        #plt.imshow(image)
        logits = processImage(image)
        if type(logits) == np.ndarray:
            arduino.write(bytes([FACE_DETECTED]))
            arduino.write(bytes(logits[0:-1]))
            while arduino.inWaiting() == 0:
                pass
            print(arduino.read_until().decode().rstrip())

            arduino.write(bytes([TRAIN_2]))
            while arduino.inWaiting() == 0:
                pass
            print(arduino.read_until().decode())
            i+=1

        else:
            arduino.write(bytes([FACE_NOT_DETECTED]))
            while arduino.inWaiting() == 0:
                pass
            print(arduino.read_until().decode())
        

Starting capture
Image captured
Reading 12296 bytes from Arducam
Finished reading
FINISHED

No Face Detected

Starting capture
Image captured
Reading 16392 bytes from Arducam
Finished reading
FINISHED

Logits Received
Example added for class: 1

Starting capture
Image captured
Reading 15368 bytes from Arducam
Finished reading
FINISHED

Logits Received
Example added for class: 1

Starting capture
Image captured
Reading 16392 bytes from Arducam
Finished reading
FINISHED

Logits Received
Example added for class: 1

Starting capture
Image captured
Reading 16392 bytes from Arducam
Finished reading
FINISHED

Logits Received
Example added for class: 1

Starting capture
Image captured
Reading 16392 bytes from Arducam
Finished reading
FINISHED

Logits Received
Example added for class: 1

Starting capture
Image captured
Reading 15368 bytes from Arducam
Finished reading
FINISHED

Logits Received
Example added for class: 1

Starting capture
Image captured
Reading 17416 bytes from Arducam
Finished 

KeyboardInterrupt: 

In [61]:
arduino.write(bytes([TRAIN_1]))
while arduino.inWaiting() == 0:
    pass
print(arduino.read_all().decode())

Example added for class: 0



In [33]:
arduino.write(bytes([TRAIN_2]))
while arduino.inWaiting() == 0:
    pass
print(arduino.read_all().decode())

Example added for class: 1



In [10]:
arduino.write(bytes([CLASSIFY]))
while arduino.inWaiting() == 0:
    pass
print(arduino.read_all().decode())

Classification Success:1



In [1]:
# CLASSIFY
while True:
    arduino.write(bytes([CAPTURE_IMAGE]))
    msgs = 0
    while True:
        try:
            msg = arduino.read_until().decode().rstrip()
            print(msg)
            if (msg == "Camera Not Initialized" or msg == "ReadData failed" or msg == "Finished reading"):
                break
        except:
            pass
    if (msg == "Camera Not Initialized" or msg == "ReadData failed"): break;
    if msg == "Finished reading":
        print('FINISHED')
        image = image_stream(arduino)
        #plt.imshow(image)
        logits = processImage(image)
        if type(logits) == np.ndarray:
            arduino.write(bytes([FACE_DETECTED]))
            arduino.write(bytes(logits[0:-1]))
            while arduino.inWaiting() == 0:
                pass
            print(arduino.read_until().decode().rstrip())

            arduino.write(bytes([CLASSIFY]))
            try:
                print(arduino.read_until().decode())
            except:
                pass

        else:
            arduino.write(bytes([FACE_NOT_DETECTED]))
            while arduino.inWaiting() == 0:
                pass
            print(arduino.read_until().decode())


NameError: name 'arduino' is not defined

In [ ]:
print(time.time())

In [39]:
arduino.read_all()

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [42]:
print(arduino.inWaiting())

256


In [31]:
print(length)

NameError: name 'length' is not defined